In [19]:
import numpy as np
import pandas as pd
import os
os.chdir('/Users/toshan/dev/pysal/pysal/contrib/spint')
from gravity import Gravity, Production, Attraction, Doubly, BaseGravity
import statsmodels.formula.api as smf
from statsmodels.api import families

import pysal

os.chdir('/Users/toshan/dev/pysal/pysal/weights')
from spintW import ODW

In [26]:
austria = pd.read_csv('http://dl.dropbox.com/u/8649795/AT_Austria.csv')
!austria = austria[austria['Origin'] != austria['Destination']]
f = austria['Data'].values
o = austria['Origin'].values
d = austria['Destination'].values
dij = austria['Dij'].values
o_vars = austria['Oi2007'].values
d_vars = austria['Dj2007'].values

/bin/sh: austria: command not found


In [27]:
w = pysal.weights.lat2W(3,3)
ODw = ODW(w,w)

In [28]:
y = f.reshape((-1,1))
x = np.hstack([o_vars.reshape((-1,1)), d_vars.reshape((-1,1)), dij.reshape((-1,1))])

In [29]:
model = ml_lag.ML_Lag(y, x, ODw)

//anaconda/lib/python2.7/site-packages/scipy/optimize/_minimize.py:596: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)


In [23]:
ODw.full()[0].shape

(81, 81)

In [24]:
ODw.n

81

In [25]:
y.shape

(72, 1)

In [30]:
model.betas

array([[ -5.45317503e+02],
       [  8.49628168e-02],
       [  8.41943053e-02],
       [ -3.23718991e+00],
       [  4.91637163e-01]])

In [31]:
model.std_err

array([  6.05869979e+02,   2.76482550e-02,   2.59612400e-02,
         1.81312140e+00,   1.56708467e-01])

In [32]:
model.z_stat

[(-0.90005697963824027, 0.36808992858846545),
 (3.0729902102679643, 0.0021192543996608137),
 (3.2430771866907935, 0.0011824617211379711),
 (-1.7854236980046085, 0.074192617586182438),
 (3.13727249558892, 0.0017052755827572379)]

In [33]:
model.pr2

0.3663580261384615